# Install libraries

In [ ]:
!pip install ray[tune]
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install wandb
!pip install sentencepiece

# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

# Import libraries

In [ ]:
from datetime import datetime
from dataclasses import dataclass, asdict, field
from typing import Union
from pydantic import BaseModel
import random
import os
import shutil

import numpy as np
import torch
import wandb
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    IntervalStrategy,
    PreTrainedModel,
    Trainer,
    TrainingArguments,
    get_linear_schedule_with_warmup,
    set_seed
)

import evaluate
import time

import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings("ignore")

import logging
logging.disable(logging.WARNING)

import inspect
from google.colab import auth
auth.authenticate_user()



wandb.login(key="14b51e44b9f1be25514c3281f4dbcabc367c86c2")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# !wandb login --relogin

In [ ]:
# !sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
# !sudo apt update >/dev/null 2>&1
# !sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
# !google-drive-ocamlfuse
# !sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
# !xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
# %cd /content
# !mkdir drive
# %cd drive
# !mkdir "MyDrive"
# !google-drive-ocamlfuse "/content/drive/MyDrive"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mount a Cloud Storage bucket or location, without the gs:// prefix.
mount_path = "x01-coliee_dir"  # or a location like "my-bucket/path/to/mount"
gs_local_path = f"{mount_path}"

!mkdir -p {gs_local_path}
!gcsfuse --implicit-dirs {mount_path} {gs_local_path}

{"time":"06/11/2023 12:26:23.723615","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /content/x01-coliee_dir\n"}


In [ ]:
data_local_path = "drive/MyDrive"
model_local_path = "models"

# Run Model

In [ ]:
# track time
start_time = time.time()

def get_key_from_value(d, value):
    for key, val in d.items():
        if val == value:
            return key
    return None

feature_dict = {
    "SENTENCE_LENGTH": ['hyp_length'],
    "WORD_OVERLAP": ['overlap', 'is_word_overlap'],
    "HAS_CONTRADICTION_WORDS": ['has_negation'],
    "SUBSEQUENCE_HEURISTICS": ['is_subsequence_heuristic'],
    "ALL": 'all',
    "NONE": "None"
}

models = {
    'BERT_BASE': 'bert-base-uncased',
    'BERT_BASE_MNLI': 'gchhablani/bert-base-cased-finetuned-mnli',
    'ROBERTA_BASE': 'roberta-base',
    'ROBERTA_BASE_MNLI': 'textattack/roberta-base-MNLI',
    'LEGAL_BERT': 'nlpaueb/legal-bert-base-uncased',
    'ELECTRA_BASE_MNLI': 'howey/electra-base-mnli',
    'DEBERTA_BASE_NLI': 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli'
}

#X01- Version --> changes --> features are added in the hyp sentence as strings using SEP token
# MAKE CHANGES TO THE CONFIG PARAMETERS HERE

# dataset_year: int = 2020 # CHANGE the dataset here
SEED = 42
DATA_TYPE = "Aug"
MODEL_TYPE = "full-context"
years = ["2018", "2019", "2020", "2021", "2022"]
feature_name = ["NONE", "ALL", "SENTENCE_LENGTH", "WORD_OVERLAP", "HAS_CONTRADICTION_WORDS", "SUBSEQUENCE_HEURISTICS"]
chosen_model = models['DEBERTA_BASE_NLI']
do_train = True

if DATA_TYPE == "Aug":
    data_file_type = "aug"
else:
    data_file_type = "normal"



for dataset_year in years:
    for feature in feature_name:
        run_num: int = 1 # CHANGE the run number here (a simple hack )
        feat_str = get_key_from_value(feature_dict, feature_dict[feature])
        @dataclass
        class GlobalConfig:
            features: list = field(default_factory=lambda: feature_dict[feature])
            device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            train_file: str = f"/content/{data_local_path}/data/task 4/train/coliee_{data_file_type}_train_{dataset_year}.csv"
            test_file: str = f"/content/{data_local_path}/data/task 4/test/coliee_test_{dataset_year}.csv"
            adv_test_file: str = f"/content/{data_local_path}/data/task 4/test/adversarial_test_set/adversarial_test_set.csv"
            train_split: float = 0.9
            num_labels: int = 2
            seed: int = SEED
            model_name: str = chosen_model # CHANGE here for models
            run_name: str = f"X01-{DATA_TYPE}-run-{dataset_year}-{run_num}-{MODEL_TYPE}-{feat_str}-features"
            max_length: int = 512 if MODEL_TYPE == "full-context" else 180 # Based on the hyp and prem sentence lengths in the dataset # CHANGE the max length here

        # Global Configuration
        global_config = GlobalConfig()

        @dataclass
        class BaseModelConfig:
            model_name: str = global_config.model_name
            num_train_epochs: int = 15 # CHANGE the epochs here
            train_batch_size: int = 8
            eval_batch_size: int = 8
            gradient_accumulation_steps: int = 4
            logging_dir: str = './logs'
            logging_steps: int = 10
            evaluation_strategy: str = "epoch"
            save_strategy: str = "epoch"
            load_best_model_at_end: bool = True
            metric_for_best_model: str = "accuracy"
            push_to_hub: bool = False
            output_dir: str = ""
            num_labels: int = 2
            log_steps: int = 10
            seed: int = global_config.seed
            warmup_steps: int = 0 # Adjust this based on your dataset size
            fp16: bool = True

        @dataclass
        class BertConfig(BaseModelConfig):
            learning_rate: float = 5e-6
            weight_decay: float = 0.01

        @dataclass
        class RoBertaConfig(BaseModelConfig):
            learning_rate: float = 5e-5
            weight_decay: float = 0.01

        @dataclass
        class ElectraConfig(BaseModelConfig):
            learning_rate: float = 5e-6
            weight_decay: float = 0.01

        @dataclass
        class DebertaConfig(BaseModelConfig):
            learning_rate: float = 3e-05
            weight_decay: float = 0.06


        # Set the seed for reproducibility
        def set_random_seed(seed: int):
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)
            np.random.seed(seed)
            random.seed(seed)
            set_seed(seed)

        set_random_seed(global_config.seed)

        # Model Configuration

        # Define a function to return model-specific configuration based on the model name
        def get_model_config(model_name: str):
            if model_name == models['BERT_BASE'] or model_name == models['BERT_BASE_MNLI'] or models['LEGAL_BERT']:
                return BertConfig()
            elif model_name == models['ROBERTA_BASE'] or model_name == models['ROBERTA_BASE_MNLI']:
                return RoBertaConfig()
            elif model_name == models['ELECTRA_BASE_MNLI']:
                return ElectraConfig()
            elif model_name == models['DEBERTA_BASE_NLI']:
                return DebertaConfig()
            else:
                raise ValueError(f"Unknown model name: {model_name}. Please check the Models dataclass for valid model names.")

        # Use the function to get the model configuration
        model_config = get_model_config(global_config.model_name)
        model_config.seed = global_config.seed

        if global_config.features == 'all':
            num_features = 5
        elif global_config.features == 'None':
            num_features = 0
        else:
            num_features = len(global_config.features)

        if num_features and MODEL_TYPE == "full-context":
            model_dir = "FullContextModelsWithFeatures"
        elif num_features and MODEL_TYPE == "hyp-only":
            model_dir = "HypOnlyModelsWithFeatures"
        elif not num_features and MODEL_TYPE == "full-context":
            model_dir = "FullContextModels"
        elif not num_features and MODEL_TYPE == "hyp-only":
            model_dir = "HypOnlyModels"

        project_name = f"X01-{DATA_TYPE}-{global_config.model_name.split('/')[1]}_{model_dir}_seed-{global_config.seed}"
        model_config.output_dir = f"/content/{model_local_path}/X01-{DATA_TYPE}-{model_dir}/seed-{global_config.seed}/{global_config.model_name}/{dataset_year}/{global_config.run_name}/"

        # initialise model and exp tracking WandB
        if do_train:
            wandb.init(
                project=project_name,
                name=global_config.run_name,
                config=asdict(model_config)
            )

        # Load the dataset
        # Data prep
        file_dict = {
            "train" : global_config.train_file,
            "test" : global_config.test_file,
        }

        if DATA_TYPE == "Aug":
            column_names = ['id', 'label', 'premise', 'hypothesis', 'labels', 'hyp_tokens','hyp_length', 'prem_tokens', 'prem_length', 'overlap', 'is_word_overlap', 'negations', 'has_negation', 'detected_subsequence', 'is_subsequence_heuristic', 'instance_type']
        else:
            column_names = ['id', 'label', 'premise', 'hypothesis', 'labels', 'hyp_tokens','hyp_length', 'prem_tokens', 'prem_length', 'overlap', 'is_word_overlap', 'negations', 'has_negation', 'detected_subsequence', 'is_subsequence_heuristic']

        dataset = load_dataset(
                'csv',
                data_files=file_dict,
                delimiter=',',
                column_names= column_names,
                skiprows=1
            )


        dataset = dataset.remove_columns('detected_subsequence')

        # view dataset
        print(dataset)
        print(dataset['train'][0])

        adv_file_dict = {
            "adv_test": global_config.adv_test_file
        }

        adv_dataset = load_dataset(
            'csv',
            data_files=adv_file_dict,
            delimiter=',',
            column_names=['id', 'label', 'premise', 'hypothesis', 'labels', 'hyp_tokens','hyp_length', 'prem_tokens', 'prem_length', 'overlap', 'is_word_overlap', 'negations', 'has_negation', 'detected_subsequence', 'is_subsequence_heuristic', 'Artefact Type', 'Adv Type'],
            skiprows=1
        )

        adv_dataset = adv_dataset.remove_columns('detected_subsequence')

        # view dataset
        print(adv_dataset)
        print(adv_dataset['adv_test'][0])


        # Tokenizer
        tokenizer = AutoTokenizer.from_pretrained(global_config.model_name, use_fast=False)

        # Custom dataset
        class CustomDataset(torch.utils.data.Dataset):
            def __init__(self, dataset, selected_features="None"):
                self.dataset = dataset
                self.selected_features = selected_features

            def __len__(self):
                return len(self.dataset)

            def extract_features(self, item):
                all_features = {
                    'hyp_length': item['hyp_length'],
                    'is_word_overlap': item['is_word_overlap'],
                    'overlap': item['overlap'],
                    'has_negations': item['has_negation'],
                    'is_subsequence_heuristic': item['is_subsequence_heuristic']
                }

                if self.selected_features == "all":
                    extracted_features = all_features
                elif self.selected_features == "None":
                    extracted_features = {}
                else:
                    extracted_features = {k: all_features[k] for k in self.selected_features if k in all_features}

                return extracted_features

            def __getitem__(self, idx):
                item = dict(self.dataset[idx])

                extracted_features = self.extract_features(item)

                prem_sentence = item['premise']
                hyp_sentence = item['hypothesis']
                if extracted_features:
                    for key, values in extracted_features.items():
                        hyp_sentence += "[SEP]" + str(values)

                # Tokenization
                inputs = tokenizer(prem_sentence, hyp_sentence, return_tensors="pt", padding='max_length', truncation='longest_first', max_length=global_config.max_length, add_special_tokens=True)
                item['input_ids'] = inputs['input_ids'].squeeze()
                item['attention_mask'] = inputs['attention_mask'].squeeze()

                return item

        # Custom hyp only dataset
        class CustomHypOnlyDataset(CustomDataset):
            def __getitem__(self, idx):
                item = dict(self.dataset[idx])

                extracted_features = self.extract_features(item)

                hyp_sentence = item['hypothesis']

                if extracted_features:
                    for key, values in extracted_features.items():
                        hyp_sentence += "[SEP]" + str(values)

                # Tokenization - Only tokenize the 'hypothesis'
                inputs = tokenizer(hyp_sentence, return_tensors="pt", padding='max_length', truncation=True, max_length=global_config.max_length, add_special_tokens=True)
                item['input_ids'] = inputs['input_ids'].squeeze()
                item['attention_mask'] = inputs['attention_mask'].squeeze()

                return item


        # Create datasets
        # Determine the splits
        test_size = (1 - global_config.train_split)
        train_data_split = dataset['train'].train_test_split(test_size=test_size, seed=42) # seed is set to be constant for producing same splits every run.

        # Split the dataset
        train_datasets = train_data_split['train']
        eval_datasets = train_data_split['test']
        test_datasets = dataset['test']
        adv_test_datasets = adv_dataset['adv_test']


        # num_train = int(global_config.train_split * len(dataset['train']))
        # num_eval = len(dataset['train']) - num_train

        # # Split the dataset
        # train_datasets = dataset['train'].select(range(num_train))
        # eval_datasets = dataset['train'].select(range(num_train, num_train + num_eval))
        # test_datasets = dataset['test']

        if MODEL_TYPE == "full-context":
            print("Entering Full Context Data prep")
            train_data = CustomDataset(train_datasets, selected_features=global_config.features)
            eval_data = CustomDataset(eval_datasets, selected_features=global_config.features)
            test_data = CustomDataset(test_datasets, selected_features=global_config.features)
            adv_test_data = CustomDataset(adv_test_datasets, selected_features=global_config.features)
            print("Done")
        elif MODEL_TYPE == "hyp-only":
            print("Entering Hyp Only Data prep")
            train_data = CustomHypOnlyDataset(train_datasets, selected_features=global_config.features)
            eval_data = CustomHypOnlyDataset(eval_datasets, selected_features=global_config.features)
            test_data = CustomHypOnlyDataset(test_datasets, selected_features=global_config.features)
            adv_test_data = CustomHypOnlyDataset(adv_test_datasets, selected_features=global_config.features)
            print("Done")

        # metrics
        metric = evaluate.load("accuracy")
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            return metric.compute(predictions=predictions, references=labels)

        def calculate_separate_acc(dataset, adv_type):
            tmp_dataset = dataset.filter(lambda row: row['Adv Type']==adv_type)
            accuracy = metric.compute(predictions=tmp_dataset['predictions'], references=tmp_dataset['label'])
            return accuracy

        def calculate_artefact_type_acc(dataset, artefact_type):
            tmp_dataset = dataset.filter(lambda row: row['Artefact Type']==artefact_type)
            accuracy = metric.compute(predictions=tmp_dataset['predictions'], references=tmp_dataset['label'])
            return accuracy

        # Initialize the model
        def load_model():
            model = AutoModelForSequenceClassification.from_pretrained(model_config.model_name, num_labels=global_config.num_labels, ignore_mismatched_sizes=True).to(global_config.device)
            print("Model Architecture")
            print(model)
            return model

        # Training arguments

        if do_train:
            model = load_model()
            training_args = TrainingArguments(
                output_dir=model_config.output_dir,
                per_device_train_batch_size=model_config.train_batch_size,
                per_device_eval_batch_size=model_config.eval_batch_size,
                num_train_epochs=model_config.num_train_epochs,
                logging_dir=model_config.logging_dir,
                logging_steps=model_config.logging_steps,
                evaluation_strategy=model_config.evaluation_strategy,
                save_strategy=model_config.save_strategy,
                load_best_model_at_end=model_config.load_best_model_at_end,
                metric_for_best_model=model_config.metric_for_best_model,
                push_to_hub=model_config.push_to_hub,
                fp16=model_config.fp16,
                save_total_limit = 1,
                gradient_accumulation_steps=model_config.gradient_accumulation_steps,
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_data,
                eval_dataset=eval_data,
                compute_metrics=compute_metrics,
                # callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
            )


            # Train the model
            trainer.train()

            # Evaluate the best model on the test and adversarial test dataset
            print("Normal test data")
            results = trainer.predict(test_data)
            print(trainer.evaluate(test_data))
            print("Adversarial test data")
            adv_results = trainer.predict(adv_test_data)
            print(trainer.evaluate(adv_test_data))

            wandb.finish()

        else:
            model = load_model()
            model.eval()
            test_args = TrainingArguments(
                output_dir = "./model_output",
                per_device_eval_batch_size=16,
                logging_dir="./test_logs/test",
            )

            tester = Trainer(
                model=model,
                args=test_args,
                compute_metrics=compute_metrics,
            )
            print("Normal test data")
            results = tester.predict(test_data)
            print(tester.evaluate(test_data))
            print("Adversarial test data")
            adv_results = tester.predict(adv_test_data)
            print(tester.evaluate(adv_test_data))



        ### NORMAL TEST RESULTS ###

        # Get instance level predictions
        predictions = results.predictions.argmax(1)
        test_datasets = test_datasets.add_column(name="predictions", column=predictions)
        test_datasets.to_csv(f"{model_config.output_dir}/{global_config.run_name}-instance_predictions.csv")

        # Write the Normal results to text file
        test_acc_file_path = f"{data_local_path}/aug_data_results/{DATA_TYPE}-full_context_test_accuracy_{global_config.seed}.txt"
        # Check if the file exists; if not, create it
        if not os.path.exists(test_acc_file_path):
            with open(test_acc_file_path, "w+") as f:
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---{results.metrics['test_accuracy']}\n")

        else:
            # Append the data to the file
            with open(test_acc_file_path, "a+") as f:
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---{results.metrics['test_accuracy']}\n")

        ### ADVERSARIAL TEST RESULTS ###

        # Get instance level predictions
        adv_predictions = adv_results.predictions.argmax(1)
        adv_test_datasets = adv_test_datasets.add_column(name="predictions", column=adv_predictions)
        adv_test_datasets.to_csv(f"{model_config.output_dir}/{global_config.run_name}-adversarial_instance_predictions.csv")
        against_acc = calculate_separate_acc(adv_test_datasets, "Against")
        for_acc = calculate_separate_acc(adv_test_datasets, "For")
        contradiction_word_acc = calculate_artefact_type_acc(adv_test_datasets, "Contradiction Word")
        word_overlap_acc = calculate_artefact_type_acc(adv_test_datasets, "Word Overlap")

        # Write the Adv results to text file
        adv_test_acc_file_path = f"{data_local_path}/aug_data_results/{DATA_TYPE}-{model_dir}_adversarial_test_accuracy_{global_config.seed}.txt"
        # Check if the file exists; if not, create it
        if not os.path.exists(adv_test_acc_file_path):
            with open(adv_test_acc_file_path, "w+") as f:
              f.write("*"*50)
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---ALL---{adv_results.metrics['test_accuracy']}\n")
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---AGAINST---{against_acc}\n")
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---FOR---{for_acc}\n")
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---CONTRADICTION WORD---{contradiction_word_acc}\n")
              f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---WORD OVERLAP---{word_overlap_acc}\n")
              f.write("*"*50)

        else:
            # Append the data to the file
            with open(adv_test_acc_file_path, "a+") as f:
                f.write("*"*50)
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---{adv_results.metrics['test_accuracy']}\n")
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---AGAINST---{against_acc}\n")
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---FOR---{for_acc}\n")
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---CONTRADICTION WORD---{contradiction_word_acc}\n")
                f.write(f"{global_config.model_name}---{global_config.run_name}---{dataset_year}---{global_config.seed}---WORD OVERLAP---{word_overlap_acc}\n")
                f.write("*"*50)


        ### UPLOAD TRAINED MODELS ###

        source_path = model_config.output_dir
        destination_path = f"gs://x01-coliee_dir/X01-{DATA_TYPE}-{model_dir}/seed-{global_config.seed}/{global_config.model_name}/{dataset_year}/"

        !gsutil -m cp -r {source_path} {destination_path}

        !rm -rf {model_local_path}

end_time = time.time()
print(f"Total time taken {(end_time - start_time)/60} mins")


wandb: Currently logged in as: venkateshdas05. Use `wandb login --relogin` to force relogin


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'premise', 'hypothesis', 'labels', 'hyp_tokens', 'hyp_length', 'prem_tokens', 'prem_length', 'overlap', 'is_word_overlap', 'negations', 'has_negation', 'is_subsequence_heuristic', 'instance_type'],
        num_rows: 618
    })
    test: Dataset({
        features: ['id', 'label', 'premise', 'hypothesis', 'labels', 'hyp_tokens', 'hyp_length', 'prem_tokens', 'prem_length', 'overlap', 'is_word_overlap', 'negations', 'has_negation', 'is_subsequence_heuristic', 'instance_type'],
        num_rows: 58
    })
})
{'id': 'H18-1-1', 'label': 1, 'premise': 'Article 572\nEven if the seller makes a special agreement to the effect that the seller does not warrant in the case prescribed in the main clause of Article 562, paragraph (1) or Article 565, the seller may not be released from that responsibility with respect to any fact that the seller knew but did not disclose, and with respect to any right that the seller personally creat

...

Filter:   0%|          | 0/42 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42 [00:00<?, ? examples/s]

Copying file:///content/models/X01-Aug-FullContextModelsWithFeatures/seed-5/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/2022/X01-Aug-run-2022-1-full-context-SUBSEQUENCE_HEURISTICS-features/X01-Aug-run-2022-1-full-context-SUBSEQUENCE_HEURISTICS-features-adversarial_instance_predictions.csv [Content-Type=text/csv]...
Copying file:///content/models/X01-Aug-FullContextModelsWithFeatures/seed-5/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/2022/X01-Aug-run-2022-1-full-context-SUBSEQUENCE_HEURISTICS-features/X01-Aug-run-2022-1-full-context-SUBSEQUENCE_HEURISTICS-features-instance_predictions.csv [Content-Type=text/csv]...
Copying file:///content/models/X01-Aug-FullContextModelsWithFeatures/seed-5/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/2022/X01-Aug-run-2022-1-full-context-SUBSEQUENCE_HEURISTICS-features/checkpoint-216/model.safetensors [Content-Type=application/octet-stream]...
Copying file:///content/models/X01-Aug-FullContextModelsWithFeatures/seed-5/MoritzLaurer/DeBERTa-v3-base-mnli-

In [ ]:
wandb.finish()

In [ ]:
os.path.exists(f"{data_local_path}/{DATA_TYPE}-{model_dir}_adversarial_test_accuracy_{global_config.seed}.txt")

False